## This notebook runs pysumma over WY 2019,2021, and 2022 with forcing data prepared from NLDAS on April 5, 2023

In [107]:
# import xarray as xr
# import pandas as pd
# import pysumma as ps

# import pysumma.plotting as psp
# import seaborn as sns
# import matplotlib.pyplot as plt

%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pysumma.plotting as psp
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import xarray as xr
import pandas as pd
import pysumma as ps

# function to convert summa 
def convert_time_to_summa_string(t):
    return (
        f'{t.dt.year.values[()]:04}'
        f'-{t.dt.month.values[()]:02}'
        f'-{t.dt.day.values[()]:02}'
        f' {t.dt.hour.values[()]:02}'
        f':{t.dt.minute.values[()]:02}'
    )

attrs = {
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}
}
name_lookup = {
    'airpres':  'pressure',
    'airtemp':  'air_K',
    'spechum':  'spechum',
    'windspd':  'wind_ms', #  
    'SWRadAtm': 'SW', #  , sw_mean_wm2
    'LWRadAtm': 'LW',
    'pptrate':  'precip_mmhr', # precip_mm per hr 'kg m-2 s-1' mm is equivalent to kg/m^2
}

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
#df = pd.read_csv('../summa_forcuplo_April_workspace_SW.csv')
df = pd.read_csv('/Users/ianwhidden/pysumma/pysumma/summa_met_forcing/new_040323/written_for_summa/nldas_sw_compare_DELETE.csv')

df.index = pd.DatetimeIndex(df['time_UTC'], name='time')

forcing_filename = 'uplo_station_forcing.nc'
# Adding 1 hour to account for SUMMA being period-ending
time_idx = df.index + pd.Timedelta('1H')
shape = (len(time_idx), 1, )
dims = ('time', 'hru', )
coords = {'time': time_idx}

met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 2015-10-01'
for varname, varattrs in attrs.items():
    df_name = name_lookup[varname]
    met_data[varname] = xr.DataArray(
        data=df[df_name].values.reshape(-1, 1),
        coords=coords, dims=dims, name=varname, attrs=varattrs
    )

#met_data['airtemp'] += 273.16  # Convert to Kelvin, not needed if air in K
met_data['pptrate'] /= 3600.0  # Convert to mm/s
met_data['data_step'] = xr.Variable([], 3600.0)
met_data.to_netcdf(f'./forcings/{forcing_filename}')

with open('./forcings/forcing_file_list.txt', 'w') as f:
    f.write(f"'{forcing_filename}'\n")

TypeError: [datetime.datetime(2015, 10, 1, 0, 0, tzinfo=tzoffset(None, -25200))
 datetime.datetime(2015, 10, 1, 1, 0, tzinfo=tzoffset(None, -25200))
 datetime.datetime(2015, 10, 1, 2, 0, tzinfo=tzoffset(None, -25200)) ...
 datetime.datetime(2018, 9, 29, 14, 0, tzinfo=tzoffset(None, -25200))
 datetime.datetime(2018, 9, 29, 15, 0, tzinfo=tzoffset(None, -25200))
 datetime.datetime(2018, 9, 29, 16, 0, tzinfo=tzoffset(None, -25200))]

In [ ]:
lat = 44.2072180256268
lon = -122.119450090239
elev = 1300
local_attrs = xr.open_dataset('../summa_setup_template/params/local_attributes.nc').load()
local_attrs['longitude'].values[:] = lon
local_attrs['latitude'].values[:] = lat
local_attrs['elevation'].values[:] = elev
local_attrs['tan_slope'] = 10.0
local_attrs['mHeight'] = 5.0
# 1 is evergreen needleleaf forest in the MODIFIED_IGBP_MODIS_NOAH option
# for the `vegeParTbl` decision. This can be found in the `VEGPARM.TBL` file
# 7 is Open Shrublands
# 16 is Barren or Sparsely Vegetated'
local_attrs['vegTypeIndex'].values[:] = 1
#local_attrs.to_netcdf('./params/local_attributes.nc')
local_attrs.to_netcdf('/Users/ianwhidden/pysumma/pysumma/April_summa_workspace/hjandrews_summa_setup/params/local_attributes.nc')
local_attrs

In [ ]:
!./install_local_setup.sh

summa_exe = 'summa.exe'
file_manager = './file_manager.txt'
# INITIATE (instantiate?) simiulation object
s = ps.Simulation(summa_exe, file_manager)
# Update file manager with start and end time
s.manager['simStartTime'] = '2015-10-02 00:00'
s.manager['simEndTime'] = '2018-09-28 15:00'
s.manager['tmZoneInfo'] = 'utcTime'

t0 = met_data['time'].isel(time=0)
t1 = met_data['time'].isel(time=-1)
s.manager['simStartTime'] = convert_time_to_summa_string(t0)
s.manager['simEndTime'] = convert_time_to_summa_string(t1)

In [ ]:
s.manager

## Output

In [ ]:
# OUTPUT
# Add a variable written to the output control file
s.output_control['scalarSnowDepth'] = [1, 0, 1, 0, 0, 0, 0, 0]
s.output_control['scalarSnowAlbedo'] = [1, 0, 1, 0, 0, 0, 0, 0]
s.output_control['SWRadAtm'] = [1, 0, 1, 0, 0, 0, 0, 0]
#'scalarCanopySnowUnloading' - unloading of snow from the vegetion canopy (kg m-2 s-1)

## Decisions

In [ ]:
# DECISIONS s - Parameter values are going to be much more important than the decisions!
# Andreadis et al. 2009 Includes a rapid interception increase between -3 and 0 C
# from observations of increased cohesion in warm regions.

# Get just the `snowIncept` option
#print(s.decisions['snowIncept'])
#print(s.decisions['windPrfile'])

# example of how to check available options for a decision
print(s.decisions['windPrfile'].available_options)

# Change the value of a decision example
#s.decisions['snowIncept'] = 'stickySnow'
#print(s.decisions['snowIncept'])

# Set decision options
#s.decisions['snowIncept']= 'stickySnow'

#s.decisions.set_option('tmZoneInfo', 'localTime') # Decision 3
s.decisions.set_option('soilCatTbl', 'ROSETTA') # Decision 4
s.decisions.set_option('vegeParTbl', 'MODIFIED_IGBP_MODIS_NOAH') # Decision 5
s.decisions.set_option('soilStress', 'NoahType') # Decision 6
s.decisions.set_option('stomResist', 'BallBerry') # Decision 7
s.decisions.set_option('fDerivMeth', 'analytic') # Decision 16
s.decisions.set_option('num_method', 'itertive') # Decision 15
s.decisions.set_option('LAI_method', 'monTable') # Decision 17
s.decisions.set_option('f_Richards', 'mixdform') # Decision 19
s.decisions.set_option('groundwatr', 'bigBuckt') # Decision 20
s.decisions.set_option('hc_profile', 'pow_prof') # Decision 21
s.decisions.set_option('bcUpprTdyn', 'nrg_flux') # Decision 22
s.decisions.set_option('bcLowrTdyn', 'presTemp') # Decision 23
s.decisions.set_option('bcUpprSoiH', 'liq_flux') # Decision 24
s.decisions.set_option('bcLowrSoiH', 'drainage') # Decision 25
s.decisions.set_option('veg_traits', 'CM_QJRMS1988') # Decision 26
s.decisions.set_option('rootProfil', 'powerLaw') # Decision 27
s.decisions.set_option('canopyEmis', 'difTrans') # Decision 28
s.decisions.set_option('snowIncept', 'stickySnow') # Decision 29
s.decisions.set_option('windPrfile', 'logBelowCanopy') # Decision 30
s.decisions.set_option('astability', 'louisinv') # Decision 31
s.decisions.set_option('compaction', 'anderson') # Decision 32
s.decisions.set_option('snowLayers', 'CLM_2010') # Decision 33
s.decisions.set_option('thCondSnow', 'smnv2000') # Decision 34
s.decisions.set_option('thCondSoil', 'funcSoilWet') # Decision 35
s.decisions.set_option('canopySrad', 'BeersLaw') # Decision 36
s.decisions.set_option('alb_method', 'varDecay') # Decision 37
s.decisions.set_option('spatial_gw', 'localColumn') # Decision 38
s.decisions.set_option('snowDenNew', 'hedAndPom') # Decision 39 anderson
s.decisions.set_option('snowUnload', 'windUnload') # Decision 40

# # Print all decisions options

# print(s.decisions['tmZoneInfo']) # Decision 3
# print(s.decisions['soilCatTbl']) # Decision 4
# print(s.decisions['vegeParTbl']) # Decision 5
# print(s.decisions['soilStress']) # Decision 6
# print(s.decisions['stomResist']) # Decision 7
# print(s.decisions['bbTempFunc']) # Decision 8
# print(s.decisions['bbHumdFunc']) # Decision 9
# print(s.decisions['bbElecFunc']) # Decision 10

# print(s.decisions['LAI_method']) # Decision 16 Method to determine LAI and SAI
# print(s.decisions['cIntercept']) # Decision 17
# print(s.decisions['canopyEmis']) # Decision 27
# print(s.decisions['snowIncept']) # Decision 28
# print(s.decisions['snowLayers']) # Decision 32
# print(s.decisions['thCondSnow']) # Decision 33
# print(s.decisions['snowDenNew']) # Decision 39
# print(s.decisions['snowUnload']) # Decision 40

In [ ]:
s.decisions

## File manager

## Parameters - 's.global_hru_params'

In [ ]:
# (5) Model Parameters -  LOCAL PARAMETERS (s.global_hru_params) - set parameters and view those changes AKA local_params.txt

# SNOW
# Max albedo of 0.85 comes from Andreadis et al., 2009
s.global_hru_params['albedoSootLoad'] = 0
print(s.global_hru_params['albedoSootLoad'])

print(s.global_hru_params['albedoMax'])
print(s.global_hru_params['albedoDecayRate'])
print(s.global_hru_params['albedoRefresh'])


# CANOPY
# height of top of the vegetation canopy above ground 
#s.global_hru_params['heightCanopyTop'] = 40.0
s.global_hru_params['heightCanopyTop'] = 0.0
# height of bottom of the vegetation canopy above ground surface (m)
#s.global_hru_params['heightCanopyBottom'] = 5.0
s.global_hru_params['heightCanopyBottom'] = 0.0
#s.global_hru_params['winterSAI'] = 10
# s.global_hru_params['refInterceptCapSnow'] = 100 # lower number, deeper snow

print(s.global_hru_params['refInterceptCapSnow']) # refInterceptCapSnow default [ 6.6 , 1.0 , 10.0 ] reference canopy interception capacity per unit leaf area (snow) (kg m-2)
# in line above m (kg m-2) is an empirical parameter determined based on observations of maximum interception capacity
print(s.global_hru_params['refInterceptCapRain']) # canopy interception capacity per unit leaf area (rain) (kg m-2)

# print(s.global_hru_params['scalarCanopyIceMax']) # maximum interception storage capacity for ice (kg m-2) throws an error
#print(s.global_hru_params['scalarCanopyLiqMax']) # throws an error

# print(s.global_hru_params['winterSAI'])
# print(s.global_hru_params['z0Canopy'])
# print(s.global_hru_params['zpdFraction'])
print(s.global_hru_params['heightCanopyTop'])
print(s.global_hru_params['heightCanopyBottom'])

# RADIATION
print(s.global_hru_params['Frad_direct'])
print(s.global_hru_params['Frad_vis'])

# Precipitation
#s.global_hru_params['tempCritRain'] = 
print(s.global_hru_params['tempCritRain']) # 32 F = 273.15 K
# frozen precipitation multiplier (-)
print(s.global_hru_params['frozenPrecipMultip'])
print(s.global_hru_params['refInterceptCapSnow']) # m (kg m-2) is an empirical parameter determined based on observations of maximum interception capacity

## Run model

In [ ]:
s.run('local')
s.status

In [ ]:
%reset

In [ ]:
# Write something from output to a csv

dssd = s.output['scalarSnowDepth']
df = dssd.to_dataframe()
df = df.to_csv('/Users/ianwhidden/Desktop/thesis_data/summa_output/summa_sensitivity_analysis/summa_output_sd_swe_WY16_17_18/sd_R0.csv')

dsswe = s.output['scalarSWE']
df = dsswe.to_dataframe()
df = df.to_csv('/Users/ianwhidden/Desktop/thesis_data/summa_output/summa_sensitivity_analysis/summa_output_sd_swe_WY16_17_18/summa_output/swe_R0.csv')

In [ ]:
dsswe = s.output['SWRadAtm']
df = dsswe.to_dataframe()
df = df.to_csv('/Users/ianwhidden/Desktop/thesis_data/summa_output/summa_sensitivity_analysis/summa_output_sd_swe_WY16_17_18/summa_output/sw.csv')

In [ ]:
## Plotting

In [ ]:
# PLOTS , current is snow albedo

# mass of total water on the vegetation canopy (kg m-2)
#s.output['scalarCanopyWat'].plot(label='SUMMA')

# temperature of each layer (K)
#s.output['mLayerTemp'].plot(label='SUMMA')

#albedo of the ground surface (-)
#s.output['scalarGroundAlbedo'].plot(label='SUMMA')


# Albedo commonly refers to the "whiteness" of a surface, with 0 meaning black and 1 meaning white.
#A value of 0 means the surface is a "perfect absorber" that absorbs all incoming energy.

#snow albedo for the entire spectral band (-)
#s.output['scalarSnowAlbedo'].plot(label='SUMMA')

albedo = s.output['scalarSnowAlbedo']
albedo.plot.line(x='time', color="purple")
# Change the max and min values of plot to show albedo variations
plt.ylim([0, 1])
# Set axis sizes
matplotlib.rc('xtick', labelsize=10) 
matplotlib.rc('ytick', labelsize=10)

#matplotlib.title('Snow Albedo')

#saturation vapor pressure at the temperature of vegetation canopy (Pa)
#s.output['scalarSatVP_CanopyTemp'].plot(label='SUMMA')

#saturation vapor pressure at the temperature of the ground (Pa)
#s.output['scalarSatVP_GroundTemp'].plot(label='SUMMA')

#net radiation (W m-2)
#Graph shows constant net radiation of zero
#s.output['scalarNetRadiation'].plot(label='SUMMA')


#latent heat from the ground (below canopy or non-vegetated) (W m-2)
#s.output['scalarLatHeatGround'].plot(label='SUMMA')


#s.output['scalarLAI'].plot(label='SUMMA')


In [ ]:
depth = s.output['scalarSnowDepth']
depth.plot.line(x='time',label='Snow depth')
#plt.suptitle('Snow depth')
#depth.title('Depth')
#plt.plot(s.output['scalarSnowDepth'], x='time')
#from matplotlib import pyplot as plt    

#fig = plt.figure()
#plt.plot(s.output['scalarSnowDepth'])
#plt.suptitle('Snowpack depth', fontsize=20)

#plt.xlabel('Time', fontsize=18)
#plt.ylabel('Snowpack depth m', fontsize=16)
#fig.savefig('test.jpg')

plt.xlabel('Date', fontsize=16)
plt.ylabel('Snow depth (m)', fontsize=16)

## Look at SW output over 1 day to determine if timestamp is correct

In [ ]:
time_range = slice('10-29-2016 00:00:00' , '10-30-2016 00:00:00')
ax = s.output['SWRadAtm'].isel(hru=0).sel(time=time_range)
#ax = s.output['SWRadAtm']
ax.plot()
#time_range = slice('10-29-2000', '04-30-2001')


In [ ]:
# PLOT TEMP IN FARENHEIT - prognostic (predicted by model) - scalarSurfaceTemp is the temp of the ground surface
temp = s.output['scalarSurfaceTemp']

# Convert
temp_f = 1.8*(temp-273) + 32
print(temp_f)
#s.output['scalarSurfaceTemp'].plot(label='SUMMA');
temp_f.plot(label='temp_F')

In [ ]:
s.output['scalarSurfaceTemp'].plot(label='SUMMA');

In [ ]:
# plot for SWE

s.output['scalarSWE'].plot(label='SUMMA');
#s.output['scalarSnowDepth'].plot(label='SUMMA');
plt.legend();

In [ ]:
# plot for something else
"""
s.output['scalarSurfaceTemp'].plot(label='SUMMA');
#s.output['scalarSnowDepth'].plot(label='SUMMA');
plt.legend();
"""
#s.output_control
#s.output['scalarSnowAlbedo'].plot(color='red', linewidth=2);
s.output

In [ ]:
#print(s.global_hru_params) # s.output[whichever cell/data]

# Total snow depth
depth = s.output.isel(hru=0)['iLayerHeight']
temp = s.output.isel(hru=0)['mLayerTemp']
frac_wat = s.output.isel(hru=0)['mLayerVolFracWat']

psp.layers(temp, depth, colormap='viridis', plot_soil=False, plot_snow=True);
s.output['scalarSnowDepth'].plot(color='red', linewidth=2);

# plt.xlabel('Date', fontsize=16)
# plt.ylabel('Snow depth (m)', fontsize=16)
# psp.savefig('testtemp.jpg')